# pyntcloud QuickStart 

The following notebook is a **real-world** example of using [pyntcloud](https://github.com/daavoo/pyntcloud).

---

It's our first day at the Jurassic Park Computer Vision Lab!!

We are given a 3D scene simulating the point clouds that are generated from a
lidar sensor on top of one of the Jurassic Park self-driving cars.

Here is a view of the 3D scene (I'm not very skillful at 3D modelling):

![view of scene](data/1.png)

### In order to run this tutorial locally, download the scene from [this link](https://mega.nz/#!qIZ13T7B!USyVexfYb8J3SDC_3h3HVUhFMXE2z0qUZNn3Ov4GXX0).

We will be only working with the parts of the scene that are visible from
the front of the car, in order to simulate what a real lidar sees.

**Our task** is to help the JP car chosing whether it should turn to the left, where
there is an apacible ankylosaurus, or right, where there is a hungry tyranosaurus.


---

Most of pyntcloud's functionallity can be accesed through the core class: [PyntCloud](http://pyntcloud.readthedocs.io/en/latest/PyntCloud.html).

In [ ]:
from pyntcloud import PyntCloud

And this is the only import we need for now.

## Loading 3D data

The first thing we are going to do is load the 3D data into Python, using PyntCloud's class method: [from_file](http://pyntcloud.readthedocs.io/en/latest/io.html)

In [ ]:
scene = PyntCloud.from_file("data/visible.ply")

We can get some basic information of the scene using the internal `__repr__` method:

In [ ]:
scene

This allows us to realize that the file is actually a triangular mesh. We can tell this because there are 99059 faces in mesh.

In [ ]:
scene.mesh.head()

Those lazy bastards from the desing deparment didn't convert the 3D model into a point cloud, wich is the real output from a lidar scanner.

## 3D point cloud from 3D mesh

Luckely for us, we can convert a triangular mesh into a point cloud:

In [ ]:
# sample 500000 points from the mesh
sampled_points = scene.get_sample("mesh_random_sampling",
                                  n=500000,
                                  rgb=True,
                                  normals=True)

# manually construcst a new PyntCloud with those points
scene = PyntCloud(sampled_points)

We can verify the result using the `__repr__` method mentioned above (this time using the `print` function) ...

In [ ]:
print(scene)

... Take a look at the information in points ...

In [ ]:
scene.points.head()

... And visualize the point cloud right inside the notebook!

In [ ]:
scene.plot()

## Floor segmentation

We are not interested in the points that compose the floor, so we will use [RANSAC](https://es.wikipedia.org/wiki/RANSAC) to find the plane that contains those points.

This is a good example of using the function [add_scalar_field](http://pyntcloud.readthedocs.io/en/latest/scalar_fields.html), wich will add a new column to `points` and return a `string` with the name of that column.

It's a good practice to store the strings returned `from PyntCloud.`**add_**`*` methods. As you will see later, this it quite convenient when chaining operations that require those strings as arguments.

In [ ]:
is_floor = scene.add_scalar_field("plane_fit",
                                  n_inliers_to_stop=len(scene.points)/30)

This method adds a new column to points.

In [ ]:
scene.points.head()

In this case, the name of the new scalar field ('is_plane') is as large as the name of the variable where we store it.

However you will find out that many scalar field have a more complex names, and thus is more convenient to always store them in a variable.

We can now visualize the point cloud colored with the new scalar field.

The `plot` function allows us to define some usefull keyword arguments.

In [ ]:
help(PyntCloud.plot)

In [ ]:
scene.plot(use_as_color=is_floor, cmap="RdYlGn", output_name="is_floor")

Now we can use the added scalar field to extract the non-floor parts of the scene.

In [ ]:
# creates a boolean array
not_floor = scene.points[is_floor] != 1 

In [ ]:
scene.points = scene.points.loc[not_floor].reset_index(drop=True)

In [ ]:
scene.points.head()

In [ ]:
scene.plot(output_name="without_floor")

## Clustering dinosaurs

Let's separate the points that belong to each dinosaur using a simple clustering technique called euclidean clustering. 

This is a good example of chaining `add_` operations storing the string indetifiers in variables.

First, we will add a VoxelGrid to the PyntCloud using [add_structure](http://pyntcloud.readthedocs.io/en/latest/structures.html).

The `sizes` parameter is quite important. If the distance between one query point and it's closest point in some cluster is higher than `size`, the point won't be considered part of that cluster.

In this case, we now that Dinosaurs are pretty damm big, and they are not really close, so let's set a separation of 3 metters along the 3 axis.

In [ ]:
vg_id = scene.add_structure("voxelgrid", sizes=[3,3,3])

Using the voxelgrid, we can add a new scalar field that indicates to wich cluster each point belong.

In [ ]:
clusters_id = scene.add_scalar_field("euclidean_clusters", voxelgrid=vg_id)

And visualize the scene colored according to those clusters:

In [ ]:
scene.plot(use_as_color=clusters_id, cmap="cool")

## Using Deep Learning to clasify dinosaurs

In [ ]:
from utils import clean_visualizations

In [ ]:
clean_visualizations(os.getcwd())